In [25]:
import pandas as pd
import numpy as np
import random

random.seed(142)

In [3]:
# Load data
df_faults = pd.read_csv('../dataset/ecg_faults_dataset.csv')
df_faults.head()

,Fault_ID,Town,Zone,Distance_km,Travel_time_hours,Fault_type,Repair_time_hours,Priority
0,F1,Afadzo South,Far,38.4,1.017,Pole replacement,1.00,High
1,F2,Liati,Far,20.0,0.550,Transformer maintenance,0.75,Normal
2,F3,Golokwati,Far,21.3,0.500,Transformer maintenance,0.75,High
3,F4,Agome yo,Near,15.9,0.517,Network line extension,2.00,Normal
4,F5,Logba,Far,38.3,1.000,Network line extension,2.00,Normal


#### **Simulating random assignment of 15 faults to 5 groups**
> **Strategy**: Randomly shuffle faults and assign them round-robin to groups

In [22]:
# random assignment 
num_faults = len(df_faults)
num_groups = 5

travel_time = df_faults['Travel_time_hours'].tolist()
repair_time = df_faults['Repair_time_hours'].tolist()
zones = df_faults['Zone'].tolist()

faults_indices = list(range(num_faults))
random.shuffle(faults_indices)

# Round-robin way of faults assignment
assignments = {i: [] for i in range(num_groups)}
for idx, fault_idx in enumerate(faults_indices):
    group = idx % num_groups
    assignments[group].append(fault_idx)

**Group metrics**

In [23]:
# Calculate metrics for each group
assignment_results = []
all_assignments = []

for group in range(num_groups):
    assigned_faults = assignments[group]
    total_travel = sum([travel_time[j] for j in assigned_faults])
    total_repair = sum([repair_time[j] for j in assigned_faults])
    total_time = total_travel + total_repair
    
    assignment_results.append({
        'Group': group,
        'Num_Faults': len(assigned_faults),
        'Total_Travel_hrs': round(total_travel, 3),
        'Total_Repair_hrs': round(total_repair, 3),
        'Total_Time_hrs': round(total_time, 3)
    })
    
    # Detailed assignments
    for j in assigned_faults:
        all_assignments.append({
            'Group': group,
            'Fault_ID': df_faults.loc[j, 'Fault_ID'],
            'Town': df_faults.loc[j, 'Town'],
            'Zone': zones[j],
            'Fault_Type': df_faults.loc[j, 'Fault_type'],
            'Travel_hrs': travel_time[j],
            'Repair_hrs': repair_time[j],
            'Total_hrs': travel_time[j] + repair_time[j]
        })

print("\nASSIGNMENT SUMMARY:")
for result in assignment_results:
    print(f"\nGroup {result['Group']}:")
    print(f"  Faults: {result['Num_Faults']}")
    print(f"  Travel: {result['Total_Travel_hrs']}h, Repair: {result['Total_Repair_hrs']}h")
    print(f"  Total: {result['Total_Time_hrs']}h")


ASSIGNMENT SUMMARY:

Group 0:
  Faults: 3
  Travel: 1.867h, Repair: 3.75h
  Total: 5.617h

Group 1:
  Faults: 3
  Travel: 2.067h, Repair: 1.83h
  Total: 3.897h

Group 2:
  Faults: 3
  Travel: 1.601h, Repair: 6.33h
  Total: 7.931h

Group 3:
  Faults: 3
  Travel: 2.0h, Repair: 6.33h
  Total: 8.33h

Group 4:
  Faults: 3
  Travel: 1.45h, Repair: 1.83h
  Total: 3.28h


#### **Summary for Random Assignments**

In [13]:
df_random = pd.DataFrame(all_assignments)
df_random = df_random.sort_values(['Group', 'Town'])

In [15]:
print("RANDOM ASSIGNMENTS:\n")
print(df_random.to_string(index=False))

RANDOM ASSIGNMENTS:

 Group Fault_ID         Town Zone                    Fault_Type  Travel_hrs  Repair_hrs  Total_hrs
     0       F3    Golokwati  Far       Transformer maintenance       0.500        0.75      1.250
     0       F2        Liati  Far       Transformer maintenance       0.550        0.75      1.300
     0      F12  Ve -Gbodome  Far      Transformer installation       0.667        4.00      4.667
     1       F8     Agome yo Near Cable joining and termination       0.517        0.33      0.847
     1       F5        Logba  Far        Network line extension       1.000        2.00      3.000
     1      F10          Wli  Far      Transformer installation       0.483        4.00      4.483
     2       F6 Afadzo South  Far Cable joining and termination       1.017        0.33      1.347
     2       F7       Akpafu Near Cable joining and termination       0.417        0.33      0.747
     2      F13   Ve- Kobenu Near Cable joining and termination       0.583        0.33 

In [17]:
# Performance metrics
print("RANDOM ASSIGNMENT PERFORMANCE:")

total_travel_random = sum([result['Total_Travel_hrs'] for result in assignment_results])
total_time_random = sum([result['Total_Time_hrs'] for result in assignment_results])

print(f"\nTotal travel time: {total_travel_random:.3f} hours")
print(f"Total work time: {total_time_random:.3f} hours")

RANDOM ASSIGNMENT PERFORMANCE:

Total travel time: 8.985 hours
Total work time: 29.055 hours


In [18]:
# Equity analysis
print("\nRANDOM ASSIGNMENT EQUITY:")

near_random = df_random[df_random['Zone'] == 'Near']
far_random = df_random[df_random['Zone'] == 'Far']

if len(near_random) > 0 and len(far_random) > 0:
    avg_near_random = near_random['Travel_hrs'].mean()
    avg_far_random = far_random['Travel_hrs'].mean()
    ratio_random = avg_far_random / avg_near_random
    
    print(f"\nNear zone: {avg_near_random:.3f}h avg travel")
    print(f"Far zone: {avg_far_random:.3f}h avg travel")
    print(f"Ratio: {ratio_random:.2f}")


RANDOM ASSIGNMENT EQUITY:

Near zone: 0.469h avg travel
Far zone: 0.748h avg travel
Ratio: 1.59


In [21]:
# Check constraint violations
print("CONSTRAINT VIOLATIONS:")
violations = []
for result in assignment_results:
    if result['Num_Faults'] > 3:
        violations.append(f"Group {result['Group']}: {result['Num_Faults']} faults (exceeds capacity of 3)")
    if result['Total_Time_hrs'] > 8:
        violations.append(f"Group {result['Group']}: {result['Total_Time_hrs']}h work time (exceeds 8h shift)")

if violations:
    print("\nConstraint violations found:")
    for v in violations:
        print(f"   • {v}")
else:
    print("\nNo constraint violations (random assignment was feasible)")

# Save
df_random.to_csv('../random_assignment_results.csv', index=False)

CONSTRAINT VIOLATIONS:

Constraint violations found:
   • Group 1: 8.33h work time (exceeds 8h shift)
